In [1]:
import numpy as np
import plotly.graph_objects as go
import os
import sys

In [2]:
# Add the parent directory to sys.path
sys.path.append(os.path.join(os.path.abspath(''), ".."))
from nwave import *

In [3]:
def func1(x):
    k1 = 11.0
    k2 = 7.0
    f = np.cos(x) + np.sin(k1 * x)
    dxf = k1 * np.cos(k1 * x) - np.sin(x)
    dxxf = -np.cos(x) - k1**2 * np.sin(k1 * x)
    return f, dxf, dxxf


def func2(x):
    """Simpler test for testing 'symmetry' to make sure x and y give the same values"""
    freq = (1.0 / 2.0) * np.pi
    f = np.cos(freq * x)

    # first order analytical derivative
    dxf = -freq * np.sin(freq * x)

    # second order analytical derivative
    dxxf = -freq * freq * np.cos(freq * x)

    return f, dxf, dxxf

def func3(x):
    """Test function similar to BSSN variables"""
    epsilon = 1.72345769e-5
    f = -2/(x + epsilon)
    dxf = 2 / ((x + epsilon) ** 2)
    dxxf = -4 / ((x + epsilon) ** 3)
    return f, dxf, dxxf

def func4(r, a):
    f = np.empty_like(r)
    dxf = np.empty_like(r)
    dxxf = np.empty_like(r)
    for i in range(len(r)):
        if r[i] < 0.0:
            f[i] = 2.0/(a**2 - r[i])
            dxf[i] = 2.0/(a**2 - r[i])**2
            dxxf[i] = 4.0/(a**2 - r[i])**3
        else:
            f[i] = -2.0/(a**2 + r[i])
            dxf[i] = 2.0/(a**2 + r[i])**2
            dxxf[i] = -4.0/(a**2 + r[i])**3

    return f, dxf, dxxf

In [4]:
nx0 = 12
level = 8
nx = int((2**level)*nx0 + 1)
params = { "Nx": nx, "Xmin": -10.0, "Xmax": 10.0 }

g = Grid1D(params)
dx = g.dx[0]
x = g.xi[0]

nBH = int(1/dx)
print(f"Grid size: {nx}, dx: {dx}, resolution at BH: {nBH}")

Grid1D: nx=3073, xmin=-10.0, xmax=10.0, dx=0.006510416666666667, ng=0 cell_centered=False
Grid size: 3073, dx: 0.006510416666666667, resolution at BH: 153


In [5]:
derivs = [ 
    DerivType.D1_KP4,
    DerivType.D1_ME44,
    DerivType.D1_ME642,
    DerivType.D1_JT4,
    DerivType.D1_JT6,
    DerivType.D1_JP6,
    DerivType.D1_DSQ6A,
    DerivType.D1_DSQ6B,
    DerivType.D1_DSQ6B_LEFT,
    DerivType.D1_DSQ6B_RIGHT,
    DerivType.D1_BYU_C6_Op10,
    DerivType.D1_BYU_P4_Op18,
]
dnames = [ 
    "D1_KP4",
    "D1_ME44",
    "D1_ME642",
    "D1_JT4",
    "D1_JT6",
    "D1_JP6",
    "D1_DSQ6A",
    "D1_DSQ6B",
    "D1_DSQ6B_LEFT",
    "D1_DSQ6B_RIGHT",
    "D1_BYU_C6_Op10",
    "D1_BYU_P4_Op18",
]

derivs2 = [
    DerivType.D2_ME44,
    DerivType.D2_ME642,
    DerivType.D2_JT4,
    DerivType.D2_JP6,
    DerivType.D2_DSQ6A,
    DerivType.D2_DSQ6B
]

dnames2 = [
    "D2_ME44",
    "D2_ME642",
    "D2_JT4",
    "D2_JP6",
    "D2_DSQ6A",
    "D2_DSQ6B"
]

In [6]:
f, dxf, dxxf = func4(x, 1.0e-3)

In [7]:
# create data files
d1_data = []

for dd, dn in zip(derivs, dnames):
    print(f"...constructing {dn}")
    CD = NCompactDerivative.deriv(x, dd, CFDSolve.SCIPY)
    CDxf = CD.grad(f)
    err = np.abs(CDxf - dxf)
    d1_data.append([dn, CDxf, err])

...constructing D1_KP4
...constructing D1_ME44
Method CFDSolve.SCIPY not supported for explicit derivatives of type DerivType.D1_ME44.  Using D_LU instead.
...constructing D1_ME642
Method CFDSolve.SCIPY not supported for explicit derivatives of type DerivType.D1_ME642.  Using D_LU instead.
...constructing D1_JT4
...constructing D1_JT6
...constructing D1_JP6
...constructing D1_DSQ6A
...constructing D1_DSQ6B
...constructing D1_DSQ6B_LEFT
...constructing D1_DSQ6B_RIGHT
...constructing D1_BYU_C6_Op10
...constructing D1_BYU_P4_Op18


In [8]:
fig = go.FigureWidget()
fig.layout.hovermode = 'closest'
fig.layout.hoverdistance = -1 #ensures no "gaps" for selecting sparse data
default_linewidth = 2
highlighted_linewidth_delta = 2

# just some traces with random data points  
num_of_traces = len(derivs)

for dd in d1_data:
    trace = go.Scatter(
        x=x, 
        y=dd[2], 
        mode='lines+markers', 
        name=dd[0], 
        line={ 'width': default_linewidth },
        hoverinfo='skip'
    )
    fig.add_trace(trace)

fig.add_vline(x=-0.5, line_width=1)
fig.add_vline(x=0.5, line_width=1)
fig.add_vrect(x0=-0.5, x1=0.5, line_width=0, fillcolor="gray", opacity=0.2)

# our custom event handler
def update_trace(trace, points, selector):
    # this list stores the points which were clicked on
    # in all but one trace they are empty
    if len(points.point_inds) == 0:
        return
        
    for i,_ in enumerate(f.data):
        fig.data[i]['line']['width'] = default_linewidth + highlighted_linewidth_delta * (i == points.trace_index)


# we need to add the on_click event to each trace separately       
for i in range( len(fig.data) ):
    fig.data[i].on_click(update_trace)

# Layout with log scale option
fig.update_layout(
    title='Derivative Error Plots (dxf)',
    xaxis=dict(title='x', type='linear'),  # use 'log' for log scale
    yaxis=dict(title='y', type='log'),
    hovermode='x',
    template='plotly_white'
)

# let's show the figure 
fig.show()

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'width': 2},
              'mode': 'lines+markers',
              'name': 'D1_KP4',
              'type': 'scatter',
              'uid': 'a76b1863-3939-47c7-82e3-45335c378741',
              'x': {'bdata': ('AAAAAAAAJMCrqqqqqvwjwFVVVVVV+S' ... 'VVVfkjQKyqqqqq/CNAAAAAAAAAJEA='),
                    'dtype': 'f8'},
              'y': {'bdata': ('AAAAAIAHUD0AAAAAAIpSPQAAAABAyU' ... 'AAgE43PQAAAAAA3gE9AAAAACyleT0='),
                    'dtype': 'f8'}},
             {'hoverinfo': 'skip',
              'line': {'width': 2},
              'mode': 'lines+markers',
              'name': 'D1_ME44',
              'type': 'scatter',
              'uid': 'b6e537b8-b9c9-4f9d-94cb-56dbcbc15caf',
              'x': {'bdata': ('AAAAAAAAJMCrqqqqqvwjwFVVVVVV+S' ... 'VVVfkjQKyqqqqq/CNAAAAAAAAAJEA='),
                    'dtype': 'f8'},
              'y': {'bdata': ('AAAAAMDePD0AAAAAACsYPQAAAAAA+A' ... 'AAAFoBPSoNili2gaQ//qb1AuF6pD8

In [9]:
# create data files
d2_data = []

for dd, dn in zip(derivs2, dnames2):
    print(f"...constructing {dn}")
    CD = NCompactDerivative.deriv(x, dd, CFDSolve.SCIPY)
    CDxxf = CD.grad(f)
    err = np.abs(CDxxf - dxxf)
    d2_data.append([dn, CDxxf, err])

...constructing D2_ME44
Method CFDSolve.SCIPY not supported for explicit derivatives of type DerivType.D2_ME44.  Using D_LU instead.
...constructing D2_ME642
Method CFDSolve.SCIPY not supported for explicit derivatives of type DerivType.D2_ME642.  Using D_LU instead.
...constructing D2_JT4
...constructing D2_JP6
...constructing D2_DSQ6A
...constructing D2_DSQ6B


In [10]:
fig = go.FigureWidget()
fig.layout.hovermode = 'closest'
fig.layout.hoverdistance = -1 #ensures no "gaps" for selecting sparse data
default_linewidth = 2
highlighted_linewidth_delta = 2

# just some traces with random data points  
num_of_traces = len(derivs)

for dd in d2_data:
    trace = go.Scatter(
        x=x, 
        y=dd[2], 
        mode='lines+markers', 
        name=dd[0], 
        line={ 'width': default_linewidth },
        hoverinfo='skip'
    )
    fig.add_trace(trace)

fig.add_vline(x=-0.5, line_width=1)
fig.add_vline(x=0.5, line_width=1)
fig.add_vrect(x0=-0.5, x1=0.5, line_width=0, fillcolor="gray", opacity=0.2)

# our custom event handler
def update_trace(trace, points, selector):
    # this list stores the points which were clicked on
    # in all but one trace they are empty
    if len(points.point_inds) == 0:
        return
        
    for i,_ in enumerate(f.data):
        fig.data[i]['line']['width'] = default_linewidth + highlighted_linewidth_delta * (i == points.trace_index)


# we need to add the on_click event to each trace separately       
for i in range( len(fig.data) ):
    fig.data[i].on_click(update_trace)

# Layout with log scale option
fig.update_layout(
    title='Derivative Error Plots (dxxf)',
    xaxis=dict(title='x', type='linear'),  # use 'log' for log scale
    yaxis=dict(title='y', type='log'),
    hovermode='x',
    template='plotly_white'
)

# let's show the figure 
fig.show()

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'width': 2},
              'mode': 'lines+markers',
              'name': 'D2_ME44',
              'type': 'scatter',
              'uid': '478c13f7-30b3-486e-9988-b1acec92b981',
              'x': {'bdata': ('AAAAAAAAJMCrqqqqqvwjwFVVVVVV+S' ... 'VVVfkjQKyqqqqq/CNAAAAAAAAAJEA='),
                    'dtype': 'f8'},
              'y': {'bdata': ('AAAA8AicuT0AAAAAbOtqPQAAAAAeI4' ... 'AA/Tt7PQAAAIAZxY09AAAAAFyOhD0='),
                    'dtype': 'f8'}},
             {'hoverinfo': 'skip',
              'line': {'width': 2},
              'mode': 'lines+markers',
              'name': 'D2_ME642',
              'type': 'scatter',
              'uid': '674e762c-cd1e-4364-a831-f0c2c9d36b94',
              'x': {'bdata': ('AAAAAAAAJMCrqqqqqvwjwFVVVVVV+S' ... 'VVVfkjQKyqqqqq/CNAAAAAAAAAJEA='),
                    'dtype': 'f8'},
              'y': {'bdata': ('AABkXvYXVD4AAEDemj0dPgAAAAAeI4' ... 'AA/Tt7PQAAAG5dQx0+AACE4+EWV

In [11]:
x_bh = 0.0
dx_bh = 0.15

d3_data = []

for dd, dn in zip(derivs, dnames):
    print(f"...constructing {dn}")
    CD = NCompactDerivative.bh_deriv(x, dd, DerivType.D1_ME44, CFDSolve.SCIPY, x_bh, dx_bh)
    CDxf = CD.grad(f)
    err = np.abs(CDxf - dxf)
    d3_data.append([dn, CDxf, err])

...constructing D1_KP4
...constructing D1_ME44
Method CFDSolve.SCIPY not supported for explicit derivatives of type DerivType.D1_ME44.  Using D_LU instead.
...constructing D1_ME642
Method CFDSolve.SCIPY not supported for explicit derivatives of type DerivType.D1_ME642.  Using D_LU instead.
...constructing D1_JT4
...constructing D1_JT6
...constructing D1_JP6
...constructing D1_DSQ6A
...constructing D1_DSQ6B
...constructing D1_DSQ6B_LEFT
...constructing D1_DSQ6B_RIGHT
...constructing D1_BYU_C6_Op10
...constructing D1_BYU_P4_Op18


In [12]:
fig = go.FigureWidget()
fig.layout.hovermode = 'closest'
fig.layout.hoverdistance = -1 #ensures no "gaps" for selecting sparse data
default_linewidth = 2
highlighted_linewidth_delta = 2

# just some traces with random data points  
num_of_traces = len(derivs)

for dd in d3_data:
    trace = go.Scatter(
        x=x, 
        y=dd[2], 
        mode='lines+markers', 
        name=dd[0], 
        line={ 'width': default_linewidth },
        hoverinfo='skip'
    )
    fig.add_trace(trace)

fig.add_vline(x=-0.5, line_width=1)
fig.add_vline(x=0.5, line_width=1)
fig.add_vrect(x0=-0.5, x1=0.5, line_width=0, fillcolor="gray", opacity=0.2)

# our custom event handler
def update_trace(trace, points, selector):
    # this list stores the points which were clicked on
    # in all but one trace they are empty
    if len(points.point_inds) == 0:
        return
        
    for i,_ in enumerate(f.data):
        fig.data[i]['line']['width'] = default_linewidth + highlighted_linewidth_delta * (i == points.trace_index)


# we need to add the on_click event to each trace separately       
for i in range( len(fig.data) ):
    fig.data[i].on_click(update_trace)

# Layout with log scale option
fig.update_layout(
    title='Masked BH Derivative Error Plots (dxf)',
    xaxis=dict(title='x', type='linear'),  # use 'log' for log scale
    yaxis=dict(title='y', type='log'),
    hovermode='x',
    template='plotly_white'
)

# let's show the figure 
fig.show()

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'width': 2},
              'mode': 'lines+markers',
              'name': 'D1_KP4',
              'type': 'scatter',
              'uid': '350bb0ee-279a-445b-a237-b35d37d49cea',
              'x': {'bdata': ('AAAAAAAAJMCrqqqqqvwjwFVVVVVV+S' ... 'VVVfkjQKyqqqqq/CNAAAAAAAAAJEA='),
                    'dtype': 'f8'},
              'y': {'bdata': ('AAAAAIAHUD0AAAAAAIpSPQAAAABAyU' ... 'AAgE43PQAAAAAA3gE9AAAAAISleT0='),
                    'dtype': 'f8'}},
             {'hoverinfo': 'skip',
              'line': {'width': 2},
              'mode': 'lines+markers',
              'name': 'D1_ME44',
              'type': 'scatter',
              'uid': '1390cebc-824a-4806-ae2a-aa985585a98e',
              'x': {'bdata': ('AAAAAAAAJMCrqqqqqvwjwFVVVVVV+S' ... 'VVVfkjQKyqqqqq/CNAAAAAAAAAJEA='),
                    'dtype': 'f8'},
              'y': {'bdata': ('AAAAAMDePD0AAAAAACsYPQAAAAAA+A' ... 'AAAFoBPSoNili2gaQ//qb1AuF6pD8